# Mentorship ETL Challenge

The Challenge:
Collect the data from https://reqres.in/api/products and from the key "data" save in a MySQL table (id,name,year and color).

In [1]:
import mysql.connector as mysql
import pandas as pd
import requests
from requests.exceptions import HTTPError
import sqlalchemy

Extraction out of ReqresAPI.

In [2]:
try:
    response = requests.get('https://reqres.in/api/products')
    jsonResponse = response.json()
    print(f'Entire JSON response: {jsonResponse}')

except HTTPError as httpError:
    print(f'HTTP Error occored: {httpError}.')
except Exception as err:
    print(f'Other error occored: {err}.')

Entire JSON response: {'page': 1, 'per_page': 6, 'total': 12, 'total_pages': 2, 'data': [{'id': 1, 'name': 'cerulean', 'year': 2000, 'color': '#98B2D1', 'pantone_value': '15-4020'}, {'id': 2, 'name': 'fuchsia rose', 'year': 2001, 'color': '#C74375', 'pantone_value': '17-2031'}, {'id': 3, 'name': 'true red', 'year': 2002, 'color': '#BF1932', 'pantone_value': '19-1664'}, {'id': 4, 'name': 'aqua sky', 'year': 2003, 'color': '#7BC4C4', 'pantone_value': '14-4811'}, {'id': 5, 'name': 'tigerlily', 'year': 2004, 'color': '#E2583E', 'pantone_value': '17-1456'}, {'id': 6, 'name': 'blue turquoise', 'year': 2005, 'color': '#53B0AE', 'pantone_value': '15-5217'}], 'support': {'url': 'https://reqres.in/#support-heading', 'text': 'To keep ReqRes free, contributions towards server costs are appreciated!'}}


Visualizing the result

In [3]:
data = jsonResponse['data']
data

[{'id': 1,
  'name': 'cerulean',
  'year': 2000,
  'color': '#98B2D1',
  'pantone_value': '15-4020'},
 {'id': 2,
  'name': 'fuchsia rose',
  'year': 2001,
  'color': '#C74375',
  'pantone_value': '17-2031'},
 {'id': 3,
  'name': 'true red',
  'year': 2002,
  'color': '#BF1932',
  'pantone_value': '19-1664'},
 {'id': 4,
  'name': 'aqua sky',
  'year': 2003,
  'color': '#7BC4C4',
  'pantone_value': '14-4811'},
 {'id': 5,
  'name': 'tigerlily',
  'year': 2004,
  'color': '#E2583E',
  'pantone_value': '17-1456'},
 {'id': 6,
  'name': 'blue turquoise',
  'year': 2005,
  'color': '#53B0AE',
  'pantone_value': '15-5217'}]

## Transformation Area

Transfering JSON data into a Pandas DataFrame.

In [4]:
df = pd.DataFrame.from_dict(data)
df

,id,name,year,color,pantone_value
0,1,cerulean,2000,#98B2D1,15-4020
1,2,fuchsia rose,2001,#C74375,17-2031
2,3,true red,2002,#BF1932,19-1664
3,4,aqua sky,2003,#7BC4C4,14-4811
4,5,tigerlily,2004,#E2583E,17-1456
5,6,blue turquoise,2005,#53B0AE,15-5217


In [6]:
df = df.drop(columns = ['pantone_value'])

In [7]:
df

,id,name,year,color
0,1,cerulean,2000,#98B2D1
1,2,fuchsia rose,2001,#C74375
2,3,true red,2002,#BF1932
3,4,aqua sky,2003,#7BC4C4
4,5,tigerlily,2004,#E2583E
5,6,blue turquoise,2005,#53B0AE


## Load to MySQL Database

Connecting with localhost to create the database

In [8]:
db = mysql.connect(
    host = 'localhost',
    user = 'root',
    password = '*Saltlake5'
)

Creating an instance of a cursor

In [9]:
cursor = db.cursor()

Creating a database for this project

In [45]:
cursor.execute('CREATE DATABASE mentroshipchallenge')

Verify if the database was successfully created.

In [10]:
cursor.execute('SHOW DATABASES;')
databases = cursor.fetchall()
for database in databases:
    print(database)

('call_center',)
('datacamp',)
('grading_system',)
('information_schema',)
('mentroshipchallenge',)
('mysql',)
('natural_gas',)
('performance_schema',)
('record_store',)
('school_record',)
('sys',)
('titanic',)
('vote',)


Utilyzing SQLAlchemy to transform a Pandas DataFrame into a MySQL table.

In [11]:
database_username = 'root'
database_password = '*Saltlake5'
database_ip       = '127.0.0.1'
database_name     = 'mentroshipchallenge'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_recycle=1, pool_timeout=57600).connect()

df.to_sql(con=database_connection, name='challenge', if_exists='append',chunksize=100)
database_connection.close()

Showing all the tables in a database.

In [13]:
#connecting to the database
db = mysql.connect(
    host='127.0.0.1',
    user='root',
    password='*Saltlake5',
    database = 'mentroshipchallenge'
)
#created a cursor instance for the database
cursor = db.cursor()
#definig query
query = 'SHOW TABLES;'
#execute query
cursor.execute(query)
#fetching all the results from the cursor object
tables = cursor.fetchall()

#showing the data
for table in tables:
    print(table)

('challenge',)


Getting all records from table

In [14]:
#Defining the query
query = 'SELECT *FROM challenge;'
#getting records from the table
cursor.execute(query)
#fetching all records from the cursor object
records = cursor.fetchall()

#showing the data
for record in records:
    print(record)

(0, 1, 'cerulean', 2000, '#98B2D1')
(1, 2, 'fuchsia rose', 2001, '#C74375')
(2, 3, 'true red', 2002, '#BF1932')
(3, 4, 'aqua sky', 2003, '#7BC4C4')
(4, 5, 'tigerlily', 2004, '#E2583E')
(5, 6, 'blue turquoise', 2005, '#53B0AE')


## Performing Queries on Data

1: Get the name of all the colors where the year is higher or equal to 2003.

In [16]:
#defining the query
query = 'SELECT name FROM challenge WHERE year >= 2003;'
#getting records from the table
cursor.execute(query)
#fetching all records from cursor
records = cursor.fetchall()

#showing the data
for record in records:
    print(record)

('aqua sky',)
('tigerlily',)
('blue turquoise',)


2: Provide a count of all the records and a mean of the years available in this table.

In [17]:
#defining the query
query = 'WITH f1 AS(SELECT COUNT(id) FROM challenge), f2 AS (SELECT);'
#getting records from the table
cursor.execute(query)
#fetching all records from cursor
result = cursor.fetchall()

print(result)

[(6,)]
